In [1]:
import tensorflow as tf

2022-08-25 13:13:18.381105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-25 13:13:18.381143: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from numpy import gradient


class LinerRegression(tf.Module):
  def __init__(self, name=None):
    super(LinerRegression,self).__init__(name=name)
    self.w = tf.Variable(tf.random.uniform([1],-1.0,1.0),name="w")
    self.b = tf.Variable(tf.zeros([1]),name="b")
    self.optimizer = tf.optimizers.SGD(0.5)
  
  @tf.function
  def __call__(self,x):
    y_hat = self.w * x + self.b
    return  y_hat

  @tf.function
  def get_w(self):
    return {"output":self.w}
  
  @tf.function
  def get_b(self):
    return {"output":self.b}
  
  @tf.function
  def train(self,x,y):
    with tf.GradientTape() as tape:
      y_hat = self(x)
      loss_value = tf.reduce_mean(tf.square(y_hat - y))
    gradients = tape.gradient(loss_value,self.trainable_variables)
    _ = self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
    return {"train":loss_value}
model = LinerRegression()




2022-08-25 13:38:43.522308: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-25 13:38:43.522335: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-25 13:38:43.522355: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (morio-H87M-S01): /proc/driver/nvidia/version does not exist
2022-08-25 13:38:43.524022: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
x = tf.TensorSpec([None],tf.float32,name="x")
y = tf.TensorSpec([None],tf.float32,name="y")
train = model.train.get_concrete_function(x,y)
w = model.get_w.get_concrete_function()
b = model.get_b.get_concrete_function()

dir = "./example"
signatures = {"train":train,"w":w,"b":b}
tf.saved_model.save(model,dir,signatures=signatures)


2022-08-25 13:41:15.221663: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./example/assets
